In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression #Least Square Regression
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error

In [2]:
#Run this for zip file
#!unzip /content/BlogFeedback.zip

Read train data

In [3]:
df = pd.read_csv("/content/blogData_train.csv", header=None).dropna()
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280
0,40.30467,53.845657,0.0,401.0,15.0,15.52416,32.44188,0.0,377.0,3.0,14.044226,32.615417,0.0,377.0,2.0,34.567566,48.475178,0.0,378.0,12.0,1.479934,46.18691,-356.0,377.0,0.0,1.076167,1.795416,0.0,11.0,0.0,0.400491,1.078097,0.0,9.0,0.0,0.377559,1.07421,0.0,9.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,40.30467,53.845657,0.0,401.0,15.0,15.52416,32.44188,0.0,377.0,3.0,14.044226,32.615417,0.0,377.0,2.0,34.567566,48.475178,0.0,378.0,12.0,1.479934,46.18691,-356.0,377.0,0.0,1.076167,1.795416,0.0,11.0,0.0,0.400491,1.078097,0.0,9.0,0.0,0.377559,1.07421,0.0,9.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,40.30467,53.845657,0.0,401.0,15.0,15.52416,32.44188,0.0,377.0,3.0,14.044226,32.615417,0.0,377.0,2.0,34.567566,48.475178,0.0,378.0,12.0,1.479934,46.18691,-356.0,377.0,0.0,1.076167,1.795416,0.0,11.0,0.0,0.400491,1.078097,0.0,9.0,0.0,0.377559,1.07421,0.0,9.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,40.30467,53.845657,0.0,401.0,15.0,15.52416,32.44188,0.0,377.0,3.0,14.044226,32.615417,0.0,377.0,2.0,34.567566,48.475178,0.0,378.0,12.0,1.479934,46.18691,-356.0,377.0,0.0,1.076167,1.795416,0.0,11.0,0.0,0.400491,1.078097,0.0,9.0,0.0,0.377559,1.07421,0.0,9.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,40.30467,53.845657,0.0,401.0,15.0,15.52416,32.44188,0.0,377.0,3.0,14.044226,32.615417,0.0,377.0,2.0,34.567566,48.475178,0.0,378.0,12.0,1.479934,46.18691,-356.0,377.0,0.0,1.076167,1.795416,0.0,11.0,0.0,0.400491,1.078097,0.0,9.0,0.0,0.377559,1.07421,0.0,9.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0


In [4]:
y_train = df.iloc[:,-1:]
print(y_train)
X_train = df.iloc[:,:-1]
print(X_train.columns)

        280
0       1.0
1       0.0
2       0.0
3       1.0
4      27.0
...     ...
52392   0.0
52393   0.0
52394   0.0
52395   0.0
52396   0.0

[52397 rows x 1 columns]
Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            270, 271, 272, 273, 274, 275, 276, 277, 278, 279],
           dtype='int64', length=280)


Read test data

In [5]:
df_test = pd.read_csv("/content/blogData_test-2012.03.31.01_00.csv", header=None).dropna()
df_test.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280
0,6.004717,37.532440,0.0,347.0,0.0,2.603774,19.723646,0.0,223.0,0.0,2.566038,19.726200,0.0,223.0,0.0,4.957547,28.919914,0.0,248.0,0.0,0.037736,22.372248,-222.0,123.0,0.0,0.231132,0.984968,0.0,8.0,0.0,0.094340,0.532497,0.0,6.0,0.0,0.084906,0.525176,0.0,6.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
1,7.777778,6.160407,1.0,18.0,6.0,3.333333,3.681787,0.0,13.0,2.0,2.777778,3.937788,0.0,13.0,1.0,6.555555,4.855950,1.0,13.0,6.0,0.555556,5.717636,-10.0,13.0,1.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.064516,0.245670,0.0,1.0,0.0,0.032258,0.176685,0.0,1.0,0.0,0.032258,0.176685,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,34.589355,98.559500,0.0,713.0,2.0,14.205323,55.165180,0.0,596.0,1.0,13.764258,55.201786,0.0,596.0,0.0,28.558935,79.903670,0.0,618.0,2.0,0.441065,68.881310,-552.0,523.0,0.0,0.216730,0.791222,0.0,6.0,0.0,0.091255,0.484787,0.0,4.0,0.0,0.087452,0.481556,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,2.000000,0.000000,2.0,2.0,2.0,0.666667,0.942809,0.0,2.0,0.0,0.666667,0.942809,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
y_test = df_test.iloc[:,-1:]
print(y_test)
X_test = df_test.iloc[:,:-1]

     280
0    0.0
1    2.0
2    0.0
3    0.0
4    0.0
..   ...
209  0.0
210  0.0
211  0.0
212  3.0
213  0.0

[214 rows x 1 columns]


Linear Regression

In [7]:
# Perform k-Fold Cross-validation with k=10 for Linear Regression

k_fold = KFold(n_splits=10, random_state=42, shuffle=True)
print(k_fold)
lr = LinearRegression()
for i in range(1,11):
  lr.fit(X_train, y_train)
  scores = cross_val_score(lr, X_train, y_train, scoring='neg_mean_squared_error', cv=k_fold)
scores = np.sqrt(np.abs(scores))
print(scores)

y_pred_lr = lr.predict(X_test)

KFold(n_splits=10, random_state=42, shuffle=True)
[36.0301857  28.47499076 27.65839302 28.48179943 29.41392232 28.53173204
 27.42623916 28.79612751 31.63314979 36.68129954]


In [8]:
print("RMSE for linear Regression/ Least Square Regression:")
rmse = np.sqrt(mean_squared_error(y_test,y_pred_lr))
rmse

RMSE for linear Regression/ Least Square Regression:


40.40010127945525

Ridge Regression

In [9]:
# Ridge Regression using cross-validation

from sklearn.linear_model import RidgeCV
alphas = 10**np.linspace(10,-2,100)*0.5

ridgecv = RidgeCV(alphas = alphas, scoring = 'neg_mean_squared_error', normalize = True)
ridgecv.fit(X_train, y_train)

y_pred_ridge = ridgecv.predict(X_test)

RMSE for Ridge regression using cross validation

In [10]:
rmse = np.sqrt(mean_squared_error(y_test,y_pred_ridge))
rmse

40.5071417521448

LASSO Regression using cross-validation

In [11]:
# LASSO Regression using cross-validation
#lassoreg = Lasso(alpha=1e-10,normalize=True)
from sklearn.linear_model import LassoCV
lassoreg = LassoCV(alphas = [1e-3,1e-2,1e-1,1], cv =10, normalize = True)
lassoreg.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LassoCV(alphas=[0.001, 0.01, 0.1, 1], copy_X=True, cv=10, eps=0.001,
        fit_intercept=True, max_iter=1000, n_alphas=100, n_jobs=None,
        normalize=True, positive=False, precompute='auto', random_state=None,
        selection='cyclic', tol=0.0001, verbose=False)

Important features using LASSO

In [12]:
# Calculate feature importance using LASSO

importance = np.abs(lassoreg.coef_)
non_zero = len(np.array(np.nonzero(importance))[0])
print(non_zero)
idx_features = (-importance).argsort()[:non_zero]
#print(idx_features)
feature_names = X_train.columns
#print(feature_names)
name_features = np.array(feature_names)[idx_features]
print('Selected features: {}'.format(name_features))

43
Selected features: [216 170  84  91 221 146  76  20 194 153  68 240 220  87 136 268 144 225
   9  81  71 157 218 260  24 262 274  95 254 101  51  56  60 183 215 115
  54  11   6  53 113  50   4]


In [ ]:
y_pred = lassoreg.predict(X_test)

RMSE for LASSO


In [14]:
np.sqrt(mean_squared_error(y_test, y_pred))

40.73096579188005